In [1]:
!sudo apt update
!sudo apt -y install build-essential vim
!pip3 install networkx node2vec python-Levenshtein nodevectors emblaze pymde

import pandas as pd
import json
import networkx as nx
from node2vec import Node2Vec
from gensim.models import KeyedVectors
import numpy as np
from collections import defaultdict

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
93 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
vim is already the newest version (2:8.2.3995-1ubuntu2.17).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.


In [2]:
all_animes_df = pd.read_csv('./work/data/processed-metadata.csv')

all_animes = []
anime_title_by_id = {}
anime_id_by_title = {}
anime_ix_by_id = {}
anime_by_id = {}

i = 0
for anime in all_animes_df.itertuples(index=False):
    all_animes.append(anime)
    anime_title_by_id[anime.id] = anime.title
    anime_id_by_title[anime.title] = anime.id
    anime_ix_by_id[anime.id] = i
    anime_by_id[anime.id] = anime
    i += 1

In [3]:
all_ratings = pd.read_csv('./work/data/collected_animelists.csv')

In [4]:
# only retain the "username", "anime_id", and "my_score" columns
ratings = all_ratings[['username', 'anime_id', 'my_score']]
# only retain rows where "my_score" is not null and greater than or equal to 6
ratings = ratings[ratings['my_score'].notnull()]
ratings = ratings[ratings['my_score'] > 0]
ratings = ratings[ratings['anime_id'].isin(anime_title_by_id.keys())]
ratings['anime_ix'] = ratings['anime_id'].apply(lambda x: anime_ix_by_id[x])

def scale_rating(rating: int) -> float:
    if rating == 10:
        return 10.1
    if rating == 9:
        return 9.1
    if rating == 8:
        return 6.5
    if rating == 7:
        return 3.3
    if rating == 6:
        return 0.5
    if rating == 5:
        return -1.5
    if rating == 4:
        return -4.5
    if rating == 3:
        return -7.5
    if rating == 2:
        return -10.5
    if rating == 1:
        return -13.5
    raise ValueError("Invalid rating: {}".format(rating))

# scale ratings from score to our custom scale
ratings['my_score'] = ratings['my_score'].apply(scale_rating)
ratings = ratings[ratings['my_score'] > 0]

print("Remaining ratings:", ratings.shape[0])
print(ratings.head())

# collect python garbage
import gc
gc.collect()

Remaining ratings: 199264878
           username  anime_id  my_score  anime_ix
0  ------____------     52034       9.1     12722
1  ------____------     37403       0.5      9638
2  ------____------     22199       9.1      6393
3  ------____------     25013       3.3      6725
4  ------____------     41433       6.5     11163


17

In [5]:
# ratings_subset = ratings.sample(n=5000000)
# ratings_subset = ratings[:5000000]
ratings_subset = ratings

# Retain only ratings of anime that have >=100 ratings
ratings_subset = ratings_subset[ratings['anime_id'].isin(anime_title_by_id.keys())]
print(f"Retained {ratings_subset.shape[0]} ratings out of {ratings.shape[0]} total")

Retained 199264878 ratings out of 199264878 total


In [6]:
all_ids = set(anime_ix_by_id.keys())
all_ixs = set(anime_ix_by_id.values())

print(f"ids: {len(all_ids)}")
print(f"ixs: {len(all_ixs)}")

ids: 13647
ixs: 13647


In [7]:
from collections import defaultdict

ratings_by_username = defaultdict(list)

i = 0
for row in ratings_subset.itertuples():
    if i % 1000000 == 0:
        print(f"Processed {i} rows")
    i += 1

    rating = row.my_score
    if rating < 0:
        continue
    anime_ix = anime_ix_by_id[row.anime_id]
    ratings_by_username[row.username].append((anime_ix, rating))

Processed 0 rows
Processed 1000000 rows
Processed 2000000 rows
Processed 3000000 rows
Processed 4000000 rows
Processed 5000000 rows
Processed 6000000 rows
Processed 7000000 rows
Processed 8000000 rows
Processed 9000000 rows
Processed 10000000 rows
Processed 11000000 rows
Processed 12000000 rows
Processed 13000000 rows
Processed 14000000 rows
Processed 15000000 rows
Processed 16000000 rows
Processed 17000000 rows
Processed 18000000 rows
Processed 19000000 rows
Processed 20000000 rows
Processed 21000000 rows
Processed 22000000 rows
Processed 23000000 rows
Processed 24000000 rows
Processed 25000000 rows
Processed 26000000 rows
Processed 27000000 rows
Processed 28000000 rows
Processed 29000000 rows
Processed 30000000 rows
Processed 31000000 rows
Processed 32000000 rows
Processed 33000000 rows
Processed 34000000 rows
Processed 35000000 rows
Processed 36000000 rows
Processed 37000000 rows
Processed 38000000 rows
Processed 39000000 rows
Processed 40000000 rows
Processed 41000000 rows
Processe

In [8]:
# for each user's ratings, deduplicate.
processed_rows = 0
for username in ratings_by_username.keys():
    before_len = len(ratings_by_username[username])
    ratings_by_username[username] = list(set(ratings_by_username[username]))
    after_len = len(ratings_by_username[username])
    if before_len != after_len:
        print(f"Removed {before_len - after_len} duplicate ratings for user {username}")
    processed_rows += 1
    if processed_rows % 500000 == 0:
        print(f"Processed {processed_rows} rows")

Processed 500000 rows
Removed 4 duplicate ratings for user nan
Processed 1000000 rows


In [9]:
ratings = ratings = [(np.asarray([anime_ix for anime_ix, rating in ratings]), np.asarray([rating for anime_ix, rating in ratings])) for ratings in ratings_by_username.values()]

In [10]:
import numpy as np
from numba import njit, prange
from numba.np.ufunc import parallel

@njit(parallel=False, nopython=True)
def compute_cooccurrence_matrix(anime_count: int, ratings: list[(np.array, np.array)]) -> np.ndarray:
    cooccurrence_matrix = np.zeros((anime_count, anime_count))

    for anime_indices, ratings in ratings:
        rating_count_for_user = len(anime_indices)

        for i in range(rating_count_for_user):
            for j in range(rating_count_for_user):
                if i == j:
                    continue

                cooccurrence_matrix[anime_indices[i], anime_indices[j]] += ratings[i] * ratings[j]

    return cooccurrence_matrix

print(f"Users to process: {len(ratings)}")
cooccurrence_matrix = compute_cooccurrence_matrix(len(anime_title_by_id), ratings)

# compute_cooccurrence_matrix.parallel_diagnostics(level=4)

/opt/conda/lib/python3.11/site-packages/numba/core/decorators.py:282: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
/opt/conda/lib/python3.11/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'ratings' of function 'compute_cooccurrence_matrix'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../tmp/ipykernel_13023/3779232740.py", line 5:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Users to process: 1440214


RuntimeError: self.setup has not been called

In [11]:
cooccurrence_matrix[0][1]

4854968.439999605

In [12]:
# Save the co-occurrence matrix to compressed binary numpy file
np.savez_compressed('./work/data/cooccurrence_matrix.npz', cooccurrence_matrix)

In [13]:
cooccurrence_matrix = np.load('./work/data/cooccurrence_matrix.npz')['arr_0']

In [14]:
import json
import math

def score_num_recommendations(num_recommendations):
    return math.sqrt(num_recommendations) / 2.

def parse_extra(item):
    related = json.loads(item.related_anime)
    related = [(r['node']['id'], 0.2 if r['relation_type'] == 'character' else 40) for r in related]
    recommended = [(r['node']['id'], score_num_recommendations(r['num_recommendations'])) for r in json.loads(item.recommendations)]
    return [(anime_ix_by_id.get(id), score) for id, score in related + recommended if anime_ix_by_id.get(id) is not None]

related_by_anime_ix = defaultdict(lambda: defaultdict(float))

for item in all_animes:
    anime_ix = anime_ix_by_id.get(item.id)
    if anime_ix is None:
        continue
    related_for_item = parse_extra(item)
    entry = related_by_anime_ix[anime_ix]
    for ix, weight in related_for_item:
        entry[ix] += weight

kill_me_baby_id = 11079
kill_me_baby_ix = anime_ix_by_id[kill_me_baby_id]
[(score, all_animes[ix].title) for ix, score in related_by_anime_ix[kill_me_baby_ix].items()]

[(40.0, 'Kill Me Baby: Butsuzou Kegatte Nise Halloween'),
 (0.2, 'Kirara Fantasia'),
 (1.224744871391589, 'Lucky☆Star'),
 (1.118033988749895, 'Nichijou'),
 (1.118033988749895, 'Acchi Kocchi'),
 (1.0, 'Yuyushiki'),
 (0.8660254037844386, 'Akuma no Riddle'),
 (0.8660254037844386, 'Danshi Koukousei no Nichijou'),
 (0.8660254037844386, 'Aho Girl'),
 (0.7071067811865476, 'Gugure! Kokkuri-san'),
 (0.7071067811865476, 'Keroro Gunsou'),
 (0.7071067811865476, 'Hetalia Axis Powers')]

In [16]:
cooccurrence_matrix_wextra = cooccurrence_matrix.copy()

for anime_0_ix in range(len(cooccurrence_matrix)):
    related_for_anime0 = related_by_anime_ix[anime_0_ix]
    max_weight = cooccurrence_matrix[anime_0_ix].max()

    for ix, weight in related_for_anime0.items():
        if ix == anime_0_ix:
            continue
        cooccurrence_matrix_wextra[anime_0_ix, ix] += weight * max_weight

np.save('./work/data/cooccurrence_matrix_wextra.npy', cooccurrence_matrix_wextra)

In [17]:
cooccurrence_matrix_wextra[0][1]

603813686.4405583

In [18]:
graph = nx.Graph()

def get_topn(weights, topn):
    return sorted(enumerate(weights), key=lambda x: x[1], reverse=True)[:topn]

topn = 40

def get_extra_topn(rating_count):
    return int(max(math.pow(rating_count, 0.46) * 0.4 - 10, 0))

for anime_0_ix in range(cooccurrence_matrix.shape[0]):
    row = cooccurrence_matrix[anime_0_ix]
    top_edges = get_topn(row, topn)
    top_related_anime_ixs_set = set([ix for ix, weight in top_edges])

    top_magnitude = 0 if len(top_edges) == 0 else top_edges[0][1]
    if top_magnitude == 0:
        continue
    related_for_anime0 = related_by_anime_ix[anime_0_ix]

    for related_anime_ix, related_weight in related_for_anime0.items():
        if related_anime_ix in top_related_anime_ixs_set:
            continue
        top_edges.append((related_anime_ix, 0.))

    for anime_1_ix, base_weight in top_edges:
        extra_weight = related_for_anime0[anime_1_ix]
        # This may need to be tuned
        weight = base_weight + extra_weight * top_magnitude * 1.
        graph.add_edge(anime_0_ix, anime_1_ix, weight=weight)

print("Edges:", graph.number_of_edges())

KeyboardInterrupt: 

In [20]:
import csrgraph as cg
import nodevectors

cgraph = cg.csrgraph(graph)

/opt/conda/lib/python3.9/site-packages/networkx/linalg/graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)


In [21]:
dimensions = 10
order = 2
algo = "ggvec"

embedding_model = None
if algo == "ggvec":
    embedding_model = nodevectors.GGVec(n_components=dimensions, learning_rate=0.01, negative_ratio=0.6, verbose=True, order=order)
elif algo == "node2vec":
    embedding_model = nodevectors.Node2Vec(walklen=8, epochs=50, return_weight=2., neighbor_weight=1., n_components=dimensions, threads=14)
elif algo == "ProNE":
    embedding_model = nodevectors.ProNE(n_components=dimensions)

embeddings = embedding_model.fit_transform(cgraph)

Loss: 0.0035	:  12%|█▏        | 42/350 [00:49<06:01,  1.17s/it]

Converged! Loss: 0.0035


In [22]:
keys = [int(n) for n in cgraph.nodes()]

print(f"key length: {len(keys)}; weight length: {len(embeddings)}; anime_count: {len(all_animes)}")

fname = f"../data/embedding_{algo}_full_posonly_wextra_top{topn}_{dimensions}d_order{order}.w2v"
with open(fname, 'wt') as f:
    tab = ' '
    nl = '\n'
    f.write(f"{len(keys)}{tab}{dimensions}{nl}")
    for key, embedding in zip(keys, embeddings):
        f.write(f"{key}{tab}{tab.join(map(str, embedding))}{nl}")
fname

key length: 11898; weight length: 11898; anime_count: 11898


'../data/embedding_ggvec_full_posonly_wextra_top40_10d_order2.w2v'